# Тема 3.3 Многотабличные запросы. 
# Лекция. Запросы к нескольким таблицам

---

### Таблица 1: **mobile_sources**

**Описание:**
Эта таблица хранит информацию о том, откуда пришёл пользователь в мобильное приложение (источник привлечения).

**Структура:**
- **user_id** — Уникальный идентификатор пользователя (в формате UUID). Каждый пользователь имеет свой уникальный идентификатор, который позволяет отслеживать его поведение и действия в системе.
- **source** — Источник, из которого пришёл пользователь. Может принимать значения вроде `other`, `yandex`, `google` и др. Это поле нужно для маркетинговой аналитики, чтобы понимать эффективность разных каналов привлечения.

---

### Таблица 2: **mobile_dataset**

**Описание:**
Эта таблица содержит данные о действиях пользователей в приложении — мобильную событийную аналитику.

**Структура:**
- **event_time** — Дата и время события (формат YYYY-MM-DD HH:MM:SS.ssssss). Показывает, когда именно произошло то или иное действие пользователя в приложении.
- **event_name** — Название события. Отражает, что именно сделал пользователь (например, `advert_open` — открытие рекламного блока, `tips_show` — показ советов и т.п.).
- **user_id** — Уникальный идентификатор пользователя (совпадает с полем user_id из первой таблицы и позволяет сопоставлять пользователя с его источником).

---

## Теоретические сведения

### Вложенные запросы и подзапросы в SQL

**Вложенные запросы** (они же **подзапросы** или *subqueries*) — это запросы, которые находятся внутри других SQL-запросов. Подзапросы позволяют сначала получить промежуточный результат, а потом использовать его для получения окончательных данных. Вложенный запрос обычно заключён в круглые скобки.

Подзапросы могут использоваться в таких SQL-операторах, как:
- `SELECT`
- `FROM`
- `WHERE`
- `HAVING`

Подразделяются на:
- **Коррелированные подзапросы** — зависят от внешнего запроса
- **Некоррелированные подзапросы** — не зависят от внешнего запроса

### Основные виды подзапросов

#### 1. Подзапрос в SELECT

Может возвращать агрегированное или вычисленное значение для строк основного запроса.

**Пример:**
```sql
SELECT user_id,
       source,
       (SELECT COUNT(*)
        FROM mobile_dataset
        WHERE mobile_dataset.user_id = mobile_sources.user_id) AS events_count
FROM mobile_sources;
```
*Пояснение:* Для каждого пользователя из таблицы mobile_sources будет рассчитано количество событий в таблице mobile_dataset.

---

#### 2. Подзапрос в WHERE

Часто используется для фильтрации результатов на основании результата другого запроса.

**Пример:**
```sql
SELECT user_id, source
FROM mobile_sources
WHERE user_id IN (
    SELECT user_id
    FROM mobile_dataset
    WHERE event_name = 'advert_open'
);
```
*Пояснение:* Выберет только тех пользователей, которые открывали рекламу.

---

#### 3. Подзапрос в FROM

Позволяет использовать результат подзапроса как временную таблицу.

**Пример:**
```sql
SELECT event_name, cnt
FROM (
    SELECT event_name, COUNT(*) AS cnt
    FROM mobile_dataset
    GROUP BY event_name
) AS event_counts
WHERE cnt > 1;
```
*Пояснение:* Считает число событий каждого типа, а во внешнем запросе фильтруются те, которые произошли больше одного раза.

---

### Сравнение с JOIN

Хотя многие задачи можно решить и с помощью соединения таблиц (`JOIN`), подзапросы удобны для лаконичного решения задач, связанных с агрегациями и отбором по промежуточным вычислениям.

---

## Примеры

1. **Найти источники, откуда пришли пользователи, совершившие хотя бы одно действие:**
```sql
SELECT source
FROM mobile_sources
WHERE user_id IN (
    SELECT DISTINCT user_id
    FROM mobile_dataset
);
```
*Здесь вложенный запрос возвращает всех пользователей, которые совершили хотя бы одно действие, а внешний — источники этих пользователей.*

---

2. **Посчитать количество действий для каждого канала привлечения:**
```sql
SELECT source,
       (SELECT COUNT(*)
        FROM mobile_dataset d
        WHERE d.user_id = s.user_id) AS actions_count
FROM mobile_sources s;
```
*Промежуточная агрегация считается во вложенном SELECT.*

---

3. **Выбрать пользователей, которые открывали рекламу чаще, чем три раза:**
```sql
SELECT user_id
FROM mobile_dataset
WHERE event_name = 'advert_open'
GROUP BY user_id
HAVING COUNT(*) > 3;
```
*Это пример использования группировки, но с подзапросом можно, например, “найти источники таких пользователей”:*

```sql
SELECT source
FROM mobile_sources
WHERE user_id IN (
    SELECT user_id
    FROM mobile_dataset
    WHERE event_name = 'advert_open'
    GROUP BY user_id
    HAVING COUNT(*) > 3
);
```

---

## Заключение

Вложенные запросы — мощный инструмент для промежуточных вычислений и более гибкой фильтрации данных. Важно помнить о возможности оптимизации: если задача решается проще через JOIN, предпочтительнее использовать его, так как он обычно быстрее, особенно на больших объёмах данных, однако подзапросы часто делают код проще и читаемее при вычислении агрегатов и при условных фильтрациях.



## Теоретические сведения: `WITH ... AS` (CTE)

**Common Table Expression (CTE)** — это временная именованная таблица, определяемая внутри одного SQL-запроса с помощью конструкции `WITH ... AS`. Она помогает разбивать сложные запросы на более мелкие, переиспользовать промежуточные результаты и делать код более чистым и читаемым.

CTE определяется **перед** основным запросом и может использоваться как обычная таблица внутри этого запроса.

**Синтаксис:**
```sql
WITH имя_таблицы AS (
    SELECT ...
)
SELECT ...
FROM имя_таблицы
```

### Преимущества использования CTE (`WITH ... AS`):

- Повышает читаемость кода (разделяется логика на части)
- Упрощает написание сложных многоуровневых запросов
- Позволяет использовать одну и ту же выборку несколько раз в запросе
- Может использоваться для рекурсивных запросов (в более продвинутых случаях)

---

## Примеры на ваших таблицах

### 1\. Пример: Посчитать количество событий для каждого пользователя

**Без CTE:**
```sql
SELECT s.user_id, s.source, COUNT(d.event_name) AS events_count
FROM mobile_sources s
LEFT JOIN mobile_dataset d ON s.user_id = d.user_id
GROUP BY s.user_id, s.source
```

**С использованием CTE:**
```sql
WITH events_per_user AS (
    SELECT user_id, COUNT(*) AS events_count
    FROM mobile_dataset
    GROUP BY user_id
)
SELECT s.user_id, s.source, e.events_count
FROM mobile_sources s
LEFT JOIN events_per_user e ON s.user_id = e.user_id
```
*Здесь сначала создаётся CTE `events_per_user`, в котором считаем события по каждому пользователю, а потом объединяем результаты с таблицей источников.*

---

### 2\. Пример: Найти источники пользователей, которые открывали рекламу больше двух раз

```sql
WITH advert_users AS (
    SELECT user_id
    FROM mobile_dataset
    WHERE event_name = 'advert_open'
    GROUP BY user_id
    HAVING COUNT(*) > 2
)
SELECT s.user_id, s.source
FROM mobile_sources s
JOIN advert_users au ON s.user_id = au.user_id
```
*В CTE `advert_users` формируется список таких пользователей, во внешнем запросе к ним подтягивается их источник.*

---

### 3\. Пример: Самые популярные события в приложении

```sql
WITH events_count AS (
    SELECT event_name, COUNT(*) AS cnt
    FROM mobile_dataset
    GROUP BY event_name
)
SELECT event_name, cnt
FROM events_count
ORDER BY cnt DESC
```
*CTE `events_count` считает количество каждого события, далее эти данные используются во внешнем запросе.*

---

### 4\. Пример: Использование нескольких CTE (цепочка CTE)

Иногда удобно построить цепочку вычислений:

```sql
WITH active_users AS (
    SELECT DISTINCT user_id
    FROM mobile_dataset
),
google_users AS (
    SELECT user_id
    FROM mobile_sources
    WHERE source = 'google'
)
SELECT gu.user_id
FROM google_users gu
JOIN active_users au ON gu.user_id = au.user_id
```
*Здесь выделяются активные пользователи и пользователи из Google, а затем ищется их пересечение.*

---

## Итоги

- **CTE** (`WITH ... AS`) — мощный инструмент для структурирования SQL-запросов.
- Это альтернатива подзапросам в `FROM` и улучшает читабельность сложных запросов.
- CTE удобен для иерархичных (вложенных) и многошаговых вычислений.

Если нужны ещё примеры или объяснение других SQL-конструкций — напишите!

## Задачи
---

### 1. Получите количество пользователей, пришедших из каждого источника, с помощью подзапроса.

```sql
SELECT source, 
       (SELECT COUNT(*) FROM mobile_sources AS ms2 WHERE ms2.source = ms1.source) AS user_count
FROM mobile_sources AS ms1
GROUP BY source;
```

---

### 2. С помощью CTE найдите список пользователей, у которых вообще есть записи в mobile_dataset.

```sql
WITH present_users AS (
    SELECT DISTINCT user_id FROM mobile_dataset
)
SELECT * FROM present_users;
```

---

### 4. Получите, с помощью CTE, количество разных событий для каждого пользователя.

```sql
WITH event_types AS (
    SELECT user_id, COUNT(DISTINCT event_name) AS type_count
    FROM mobile_dataset
    GROUP BY user_id
)
SELECT * FROM event_types;
```

---

### 5. Используйте подзапрос, чтобы вывести все источники, из которых пришли пользователи, совершившие событие `tips_show`.

```sql
SELECT DISTINCT source 
FROM mobile_sources
WHERE user_id IN (
    SELECT user_id FROM mobile_dataset WHERE event_name = 'tips_show'
);
```

---

### 6. С помощью CTE найдите среднее количество событий для пользователей из каждого источника.

```sql
WITH user_event_counts AS (
    SELECT ms.source, ms.user_id, COUNT(md.event_name) AS event_count
    FROM mobile_sources ms
    LEFT JOIN mobile_dataset md ON ms.user_id = md.user_id
    GROUP BY ms.source, ms.user_id
)
SELECT source, AVG(event_count) AS avg_events
FROM user_event_counts
GROUP BY source;
```

---

### 7. Выведите с помощью подзапроса, для каждого пользователя минимальную дату события.

```sql
SELECT user_id, 
       (SELECT MIN(event_time) FROM mobile_dataset md2 WHERE md2.user_id = ms.user_id) AS first_event_time
FROM mobile_sources ms;
```

---

### 8. С помощью CTE укажите для каждого источника число уникальных событий, которые совершали его пользователи.

```sql
WITH source_events AS (
    SELECT ms.source, md.event_name
    FROM mobile_sources ms
    JOIN mobile_dataset md ON ms.user_id = md.user_id
    GROUP BY ms.source, md.event_name
)
SELECT source, COUNT(event_name) AS unique_events
FROM source_events
GROUP BY source;
```

---

### 9. По каждому пользователю с помощью подзапроса найдите количество совершённых им событий типа `advert_open`.

```sql
SELECT user_id, 
       (SELECT COUNT(*) FROM mobile_dataset md2 WHERE md2.user_id = ms.user_id AND event_name = 'advert_open') AS advert_open_count
FROM mobile_sources ms;
```

---

### 10. Для каждого источника найдите среднее число событий на пользователя через CTE, а в основном запросе выберите только те источники, где это значение больше 2.

```sql
WITH source_user_events AS (
    SELECT ms.source, ms.user_id, COUNT(md.event_name) AS cnt
    FROM mobile_sources ms
    LEFT JOIN mobile_dataset md ON ms.user_id = md.user_id
    GROUP BY ms.source, ms.user_id
), avg_per_source AS (
    SELECT source, AVG(cnt) AS avg_events
    FROM source_user_events
    GROUP BY source
)
SELECT source, avg_events
FROM avg_per_source
WHERE avg_events > 2;
```

---

### 11. Найдите для каждого источника количество пользователей, которые совершили хотя бы одно событие `tips_show` и у которых общее количество событий больше среднего по своей группе.

```sql
WITH users_with_tips AS (
    SELECT ms.source, ms.user_id, COUNT(md.event_name) AS events_cnt
    FROM mobile_sources ms
    JOIN mobile_dataset md ON ms.user_id = md.user_id
    WHERE md.event_name = 'tips_show'
    GROUP BY ms.source, ms.user_id
),
avg_events AS (
    SELECT source, AVG(events_cnt) AS avg_cnt
    FROM users_with_tips
    GROUP BY source
)
SELECT u.source, COUNT(u.user_id) as users_above_avg
FROM users_with_tips u
JOIN avg_events a ON u.source = a.source
WHERE u.events_cnt > a.avg_cnt
GROUP BY u.source;
```

---

### 12. Для каждого пользователя, совершившего хотя бы 3 события, найдите разницу между количеством событий этого пользователя и максимальным количеством событий пользователя из его источника.

```sql
WITH user_events AS (
    SELECT ms.source, ms.user_id, COUNT(md.event_name) AS cnt
    FROM mobile_sources ms
    JOIN mobile_dataset md ON ms.user_id = md.user_id
    GROUP BY ms.source, ms.user_id
    HAVING COUNT(md.event_name) >= 3
), max_per_source AS (
    SELECT source, MAX(cnt) AS mx
    FROM user_events
    GROUP BY source
)
SELECT u.user_id, u.source, u.cnt,
       (SELECT mx FROM max_per_source WHERE source = u.source) - u.cnt AS diff_from_max
FROM user_events u;
```

---

### 13. Найдите пользователей, для которых минимальный промежуток между их событиями меньше среднего минимального промежутка для пользователей их источника.

```sql
WITH user_gaps AS (
    SELECT md.user_id,
      MIN(md.event_time - LAG(md.event_time) OVER (PARTITION BY md.user_id ORDER BY md.event_time)) AS min_gap
    FROM mobile_dataset md
    GROUP BY md.user_id
),
user_source_gap AS (
    SELECT ms.source, ug.user_id, ug.min_gap
    FROM user_gaps ug JOIN mobile_sources ms ON ug.user_id = ms.user_id
),
avg_min_gap AS (
    SELECT source, AVG(EXTRACT(EPOCH FROM min_gap)) AS avg_min_gap_sec
    FROM user_source_gap
    GROUP BY source
)
SELECT u.user_id, u.source, u.min_gap
FROM user_source_gap u
JOIN avg_min_gap a ON u.source = a.source
WHERE EXTRACT(EPOCH FROM u.min_gap) < a.avg_min_gap_sec;
```

---

### 14. Для каждого источника посчитайте среднюю длину имени события у пользователей, совершивших больше событий, чем медианное количество по своему источнику.

```sql
WITH user_counts AS (
    SELECT ms.source, ms.user_id, COUNT(md.event_name) AS cnt
    FROM mobile_sources ms
    JOIN mobile_dataset md ON ms.user_id = md.user_id
    GROUP BY ms.source, ms.user_id
),
medians AS (
    SELECT source, PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY cnt) AS median_cnt
    FROM user_counts
    GROUP BY source
),
active_users AS (
    SELECT u.source, u.user_id
    FROM user_counts u
    JOIN medians m ON u.source = m.source
    WHERE u.cnt > m.median_cnt
),
user_event_lengths AS (
    SELECT au.source, LENGTH(md.event_name) AS len
    FROM active_users au
    JOIN mobile_dataset md ON au.user_id = md.user_id
)
SELECT source, AVG(len) AS avg_event_name_length
FROM user_event_lengths
GROUP BY source;
```

---

### 15. Для каждого пользователя среди тех, кто совершил свои первые два события в один и тот же день, найдите долю различий во времени между этими событиями среди всех таких пользователей.

```sql
WITH first_two AS (
    SELECT user_id, event_time,
           ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY event_time) AS rn
    FROM mobile_dataset
),
same_day_users AS (
    SELECT f1.user_id,
           f2.event_time - f1.event_time AS diff
    FROM first_two f1
    JOIN first_two f2 ON f1.user_id = f2.user_id AND f1.rn = 1 AND f2.rn = 2
    WHERE DATE(f1.event_time) = DATE(f2.event_time)
),
all_diffs AS (
    SELECT diff FROM same_day_users
)
SELECT user_id, diff,
    (SELECT COUNT(*) FROM all_diffs WHERE diff = s.diff)::FLOAT 
    / (SELECT COUNT(*) FROM all_diffs) AS share_of_diff
FROM same_day_users s;
```

---

### 16. Для каждого источника вычислите процент пользователей, которые совершили более одного типа события, среди всех пользователей этого источника.

```sql
WITH events_per_user AS (
    SELECT ms.source, ms.user_id, COUNT(DISTINCT md.event_name) AS event_types
    FROM mobile_sources ms
    LEFT JOIN mobile_dataset md ON ms.user_id = md.user_id
    GROUP BY ms.source, ms.user_id
),
user_counts AS (
    SELECT source, COUNT(*) AS total_users,
           SUM(CASE WHEN event_types > 1 THEN 1 ELSE 0 END) AS multi_event_users
    FROM events_per_user
    GROUP BY source
)
SELECT source, 
       ROUND(multi_event_users::NUMERIC / total_users * 100, 2) AS percent_multi_event
FROM user_counts;
```

---

### 17. Выведите для каждого пользователя дату его самого частого события и саму частоту этого события, если пользователь совершил хотя бы 4 события.

```sql
WITH user_event_counts AS (
    SELECT user_id, event_name, COUNT(*) AS cnt
    FROM mobile_dataset
    GROUP BY user_id, event_name
),
user_total_events AS (
    SELECT user_id, SUM(cnt) AS total
    FROM user_event_counts
    GROUP BY user_id
    HAVING SUM(cnt) >= 4
)
SELECT uec.user_id, uec.event_name, uec.cnt
FROM user_event_counts uec
JOIN user_total_events ute ON uec.user_id = ute.user_id
WHERE uec.cnt = (
    SELECT MAX(cnt)
    FROM user_event_counts
    WHERE user_id = uec.user_id
);
```

---

### 18. Найдите пользователей, совершивших событие с минимальной длиной имени события в истории этого пользователя, если длина имени этого события равна максимально короткому среди пользователей их источника.

```sql
WITH event_lengths AS (
    SELECT ms.source, md.user_id, LENGTH(md.event_name) AS len
    FROM mobile_sources ms
    JOIN mobile_dataset md ON ms.user_id = md.user_id
),
min_event_per_user AS (
    SELECT user_id, MIN(len) AS min_len
    FROM event_lengths
    GROUP BY user_id
),
min_len_per_source AS (
    SELECT el.source, MIN(u.min_len) AS min_source_len
    FROM event_lengths el
    JOIN min_event_per_user u ON el.user_id = u.user_id AND el.len = u.min_len
    GROUP BY el.source
)
SELECT el.user_id
FROM event_lengths el
JOIN min_event_per_user ue ON el.user_id = ue.user_id AND el.len = ue.min_len
JOIN min_len_per_source ms ON el.source = ms.source
WHERE ue.min_len = ms.min_source_len;
```

---

### 19. Посчитайте для каждого источника медиенную разницу во времени между последним и первым событием среди пользователей, совершивших не менее 5 событий.

```sql
WITH first_last AS (
    SELECT ms.source, md.user_id, MIN(md.event_time) AS min_time, MAX(md.event_time) AS max_time, COUNT(*) AS cnt
    FROM mobile_sources ms
    JOIN mobile_dataset md ON ms.user_id = md.user_id
    GROUP BY ms.source, md.user_id
),
filtered AS (
    SELECT source, user_id, max_time - min_time AS span
    FROM first_last WHERE cnt >= 5
)
SELECT source,
       percentile_cont(0.5) WITHIN GROUP (ORDER BY span) AS median_span
FROM filtered
GROUP BY source;
```

---

### 20. Сформируйте для каждого источника таблицу из трех колонок: общее число пользователей, доля пользователей с хотя бы одним событием, доля пользователей с более чем 10 событиями.

```sql
WITH user_events AS (
    SELECT ms.source, ms.user_id, COUNT(md.event_name) AS cnt
    FROM mobile_sources ms
    LEFT JOIN mobile_dataset md ON ms.user_id = md.user_id
    GROUP BY ms.source, ms.user_id
),
agg AS (
    SELECT source,
           COUNT(user_id) AS total_users,
           SUM(CASE WHEN cnt > 0 THEN 1 ELSE 0 END) as with_events,
           SUM(CASE WHEN cnt > 10 THEN 1 ELSE 0 END) as above_10
    FROM user_events
    GROUP BY source
)
SELECT source, total_users,
       ROUND(with_events::NUMERIC/total_users*100,2) AS percent_with_events,
       ROUND(above_10::NUMERIC/total_users*100,2) AS percent_above_10
FROM agg;
```